<a href="https://colab.research.google.com/github/sergeo/Rainfall_Workshop/blob/main/stormTrackTAHMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import os
from geopy.distance import geodesic

# Path to folder containing CSV files
folder_path = r"D:\TAHMO\TEMBO\worskhop_nairobi\distances"

# define functions
# function for gauge rainfall based on storm properties
def class_outs(r1,r2,M):
    # r1 is the distance between gauge and the storm centriod
    # r2 is the radius of the storm
    # M is the magnitude which is computed in 2 
    if r1 == 0:        
        mag = M
        classif = 'Extreme High'
    
    elif r1 < 0.5*r2:
        mag = 0.95*M
        classif = 'High'
    
    elif (0.5*r2 < r1) and r1 < r2:
        mag = 0.5*M
        classif = 'Med'
    elif r2 < r1:
        mag = 0
        classif = 'low'
    
    return classif, mag

# Get a list of CSV files in the folder containing the storm properties
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# merge CSV files containing storm properties
# Initialize an empty list to store the merged data
merged_data = []

# Loop through each CSV file and append to merged_data
for i, file in enumerate(csv_files):
    # Load CSV file into a pandas DataFrame
    df = pd.read_csv(os.path.join(folder_path, file))
    
    # Add a new column "stormName" with the value of the file name
    storm_name = os.path.splitext(file)[0]
    df['stormName'] = storm_name
    
    # Append DataFrame to merged_data
    merged_data.append(df)

# Concatenate all DataFrames in merged_data into a single DataFrame
merged_df = pd.concat(merged_data)

# Save merged DataFrame to a new CSV file
merged_df.to_csv(os.path.join(folder_path, r"D:\TAHMO\TEMBO\worskhop_nairobi\distances\Results\merged2.csv"), index=False)

# read in the merge.csv and metadata.csv files
merge_df = pd.read_csv(os.path.join(folder_path, r"D:\TAHMO\TEMBO\worskhop_nairobi\distances\Results\merged2.csv"))

metadata_df = pd.read_csv(r"D:\TAHMO\TEMBO\worskhop_nairobi\distances\station_data\metadata.csv")

# iterate over each row in merge_df and calculate distance to all locations in metadata_df
for i, row in merge_df.iterrows(): # merge_df contains storm properties
    radius_of_storm = row['Radius'] # 'Radius'
    area_of_storm = 100*row['Count'] # 100 is the area of a single pixel 10 X 10 km
    magnitude_of_storm = row['Mag'] # total rainfall in the storm (mm)
    
    # two magnitude of rainfall computations are done 
    # 1) magnitude of rainfall at a pixel derived from the raining cloud's magnitude and the total number of pixels in the cloud
    # 2)  magnitude of rainfall at a pixel derived from the raining cloud's magnitude and the area of the cloud
    # one must decide which one is used to evaluate the magnitude of the rainfall at the gauge's point
    # and also decide where the gauge's point is recieving high, medium or low rainfall
    merge_df.loc[i,'Ave_Magnitude_rainfall_per_pixel'] = row['Mag']/row['Count']
    merge_df.loc[i,'Ave_Magnitude_rainfall_per_pixel_area'] = row['Mag']/area_of_storm   
    
    storm_centroid_y,storm_centroid_x = row['lat'], row['long']
    
    for j, metadata_row in metadata_df.iterrows():        
        gauge_y_loc,gauge_x_loc = metadata_row['latitude'], metadata_row['longitude']
        distance = geodesic((storm_centroid_y,storm_centroid_x), (gauge_y_loc,gauge_x_loc)).km
        
        stcde = metadata_row['station code']
        
        # add computed distances to stirm properties dataframe
        merge_df.loc[i,stcde+' (km)'] = distance # distnace of the gauge to the centre of the storm
        
        # here magnitude of rainfall per pixel based 
        # for simplicity sake, computation of magnitude 1 is used
        # of course this needs investigation
        classification_paramters = class_outs(distance,radius_of_storm,merge_df.loc[i,'Ave_Magnitude_rainfall_per_pixel_area'])
        merge_df.loc[i,stcde+' (mm)'] = classification_paramters[1] # fraction of the magnitude of the storm based on the gauge's proximity to the storm
        merge_df.loc[i,stcde+'_class'] = classification_paramters[0] # rain classification based on gauge's proximity to storm
        
# save the merged dataframe with distances as a new csv file
merge_df.to_csv(os.path.join(folder_path, r"D:\TAHMO\TEMBO\worskhop_nairobi\distances\Results\merged_with_distances2.csv"), index=True)
